In [95]:
#dependencies 

import numpy as np
import pandas as pd

In [96]:
#import cleaned data csv
all_wells = pd.read_csv('Cleaned_Data/well_cleaned.csv')
all_wells

,Unnamed: 0,WELL_BORE_CODE,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,FLOW_KIND,WELL_TYPE
0,15,NO 15/9-F-1 C,289.42,106.35,43.34,107.36,37.94,78.94,631.47,90439.09,0.00,production,OP
1,16,NO 15/9-F-1 C,270.24,107.64,47.17,99.19,60.76,70.63,1166.46,165720.39,0.00,production,OP
2,17,NO 15/9-F-1 C,262.84,107.87,47.73,94.60,63.05,66.05,1549.81,221707.31,0.00,production,OP
3,18,NO 15/9-F-1 C,255.53,107.97,48.53,89.99,64.55,61.41,1248.70,178063.52,0.00,production,OP
4,19,NO 15/9-F-1 C,247.20,108.05,49.84,84.78,65.72,56.15,1345.78,192602.19,0.00,production,OP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5938,8923,NO 15/9-F-15 D,194.98,106.52,31.58,15.81,49.02,1.26,144.01,23201.35,203.93,production,OP
5939,8924,NO 15/9-F-15 D,194.98,106.52,31.54,15.77,48.99,1.20,145.22,23068.07,202.93,production,OP
5940,8925,NO 15/9-F-15 D,194.32,106.52,31.52,15.70,50.10,1.28,142.74,23059.68,203.84,production,OP
5941,8926,NO 15/9-F-15 D,195.21,106.51,31.52,15.61,49.84,1.20,144.46,23090.47,202.76,production,OP


# Neural Network Analysis

### AVG_CHOKE_SIZE_P, AVG_WHP_P, AVG_WHT_P, DP_CHOKE_SIZE, BORE_OIL_VOL, BORE_GAS_VOL, BORE_WAT_VOL

In [97]:
#read in data for analysis 
X1= all_wells[["AVG_CHOKE_SIZE_P","AVG_WHP_P","AVG_WHT_P", "DP_CHOKE_SIZE","BORE_OIL_VOL","BORE_GAS_VOL", "BORE_WAT_VOL"]]
y1= all_wells["AVG_DOWNHOLE_PRESSURE"].values.reshape(-1, 1)
print(X1.shape)

(5943, 7)


In [98]:
#split into test and train data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=42)

In [99]:
from sklearn.preprocessing import StandardScaler
# # Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
# # Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [100]:
#create the neural network
from tensorflow.keras.models import Sequential
model = Sequential()

In [101]:
#create the model
from tensorflow.keras.layers import Dense
number_inputs = X_train.shape[1]
number_hidden_nodes = 100

model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))
model.add(Dense(number_hidden_nodes, kernel_initializer='normal',activation='relu'))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

In [102]:
#metrics
# CREDIT: https://github.com/keras-team/keras/issues/7947
# root mean squared error (rmse) for regression (only for Keras tensors)
def rmse(y_true, y_pred):
    from keras import backend
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression  (only for Keras tensors)
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression  (only for Keras tensors)
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [103]:
#compile the model
from keras import losses

model.compile(loss="mean_absolute_error",
              optimizer="adam", metrics=[r_square, rmse])

In [104]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               800       
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 11,001
Trainable params: 11,001
Non-trainable params: 0
_________________________________________________________________


In [105]:
from keras.callbacks import EarlyStopping
es= EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=5,verbose=0, mode='min')
model.fit(
    X_test_scaled,
    y_test_scaled,
    epochs=100,
    shuffle=True,
    verbose=2,
    validation_split= .15,
    callbacks= [es]
)

Train on 1263 samples, validate on 223 samples
Epoch 1/100
1263/1263 - 1s - loss: 0.6122 - r_square: 0.3356 - rmse: 0.6122 - val_loss: 0.4624 - val_r_square: 0.5474 - val_rmse: 0.4624
Epoch 2/100
1263/1263 - 0s - loss: 0.4107 - r_square: 0.6122 - rmse: 0.4107 - val_loss: 0.3697 - val_r_square: 0.6541 - val_rmse: 0.3697
Epoch 3/100
1263/1263 - 0s - loss: 0.3148 - r_square: 0.7300 - rmse: 0.3148 - val_loss: 0.3078 - val_r_square: 0.7023 - val_rmse: 0.3078
Epoch 4/100
1263/1263 - 0s - loss: 0.2612 - r_square: 0.7992 - rmse: 0.2612 - val_loss: 0.2890 - val_r_square: 0.7084 - val_rmse: 0.2890
Epoch 5/100
1263/1263 - 0s - loss: 0.2375 - r_square: 0.8154 - rmse: 0.2375 - val_loss: 0.2829 - val_r_square: 0.7129 - val_rmse: 0.2829
Epoch 6/100
1263/1263 - 0s - loss: 0.2275 - r_square: 0.8315 - rmse: 0.2275 - val_loss: 0.2629 - val_r_square: 0.7157 - val_rmse: 0.2629
Epoch 7/100
1263/1263 - 0s - loss: 0.2212 - r_square: 0.8439 - rmse: 0.2212 - val_loss: 0.2548 - val_r_square: 0.7237 - val_rmse: 0

In [106]:
model.evaluate(X_test_scaled, y_test_scaled, batch_size=50)

1486/1486 [==============================] - 0s 22us/sample - loss: 0.1587 - r_square: 0.8642 - rmse: 0.1587


[0.15868836437749348, 0.8642269, 0.15868835]

In [107]:
#test two
from keras.callbacks import EarlyStopping
es= EarlyStopping(monitor='val_r_square', min_delta=0.0001, patience=5,verbose=0, mode='max')
model.fit(
    X_test_scaled,
    y_test_scaled,
    epochs=100,
    shuffle=True,
    verbose=2,
    validation_split= .15,
    callbacks= [es]
)

Train on 1263 samples, validate on 223 samples
Epoch 1/100
1263/1263 - 0s - loss: 0.1604 - r_square: 0.8900 - rmse: 0.1604 - val_loss: 0.1931 - val_r_square: 0.7105 - val_rmse: 0.1931
Epoch 2/100
1263/1263 - 0s - loss: 0.1619 - r_square: 0.8900 - rmse: 0.1619 - val_loss: 0.1945 - val_r_square: 0.6983 - val_rmse: 0.1945
Epoch 3/100
1263/1263 - 0s - loss: 0.1407 - r_square: 0.8958 - rmse: 0.1407 - val_loss: 0.1985 - val_r_square: 0.7006 - val_rmse: 0.1985
Epoch 4/100
1263/1263 - 0s - loss: 0.1402 - r_square: 0.9057 - rmse: 0.1402 - val_loss: 0.1935 - val_r_square: 0.7051 - val_rmse: 0.1935
Epoch 5/100
1263/1263 - 0s - loss: 0.1472 - r_square: 0.8993 - rmse: 0.1472 - val_loss: 0.1890 - val_r_square: 0.7071 - val_rmse: 0.1890
Epoch 6/100
1263/1263 - 0s - loss: 0.1469 - r_square: 0.8939 - rmse: 0.1469 - val_loss: 0.1874 - val_r_square: 0.7073 - val_rmse: 0.1874


In [108]:
model.evaluate(X_test_scaled, y_test_scaled, batch_size=50)

1486/1486 [==============================] - 0s 21us/sample - loss: 0.1453 - r_square: 0.8705 - rmse: 0.1453


[0.14532866795242716, 0.87052745, 0.14532866]

# Hyperas


In [110]:
#split into test and train data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=42)

In [111]:
def data():
    #read in data for analysis 
    all_wells = pd.read_csv('Cleaned_Data/well_cleaned.csv')
    X1= all_wells[["AVG_CHOKE_SIZE_P","AVG_WHP_P","AVG_WHT_P", "DP_CHOKE_SIZE","BORE_OIL_VOL","BORE_GAS_VOL", "BORE_WAT_VOL"]]
    y1= all_wells["AVG_DOWNHOLE_PRESSURE"].values.reshape(-1, 1)
    #split into test and train data
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=42)
    from sklearn.preprocessing import StandardScaler
    # # Create a StandardScater model and fit it to the training data
    X_scaler = StandardScaler().fit(X_train)
    y_scaler = StandardScaler().fit(y_train)
    # # Transform the training and testing data using the X_scaler and y_scaler models
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    y_train_scaled = y_scaler.transform(y_train)
    y_test_scaled = y_scaler.transform(y_test)
    
    x_train = X_train_scaled.reshape(-1,7)
    x_test = X_test_scaled.reshape(-1,7)
    y_train = y_train_scaled.reshape(-1,1)
    y_test = y_test_scaled.reshape(-1,1)
    return x_train, y_train, x_test, y_test

In [112]:
#metrics
# CREDIT: https://github.com/keras-team/keras/issues/7947
# root mean squared error (rmse) for regression (only for Keras tensors)
def rmse(y_true, y_pred):
    from keras import backend
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression  (only for Keras tensors)
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression  (only for Keras tensors)
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [14]:
# from sklearn.preprocessing import StandardScaler
# # # Create a StandardScater model and fit it to the training data
# X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)
# # # Transform the training and testing data using the X_scaler and y_scaler models
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

In [134]:
# from __future__ import print_function
# from __future__ import absolute_import, division, print_function
exec('from __future__ import absolute_import, division, print_function')
from hyperas.distributions import uniform
from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras import backend as K
# from keras.utils import print_summary
# from tensorflow.python.keras.layers import Input, Dense
# from tensorflow.python.keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation

def create_model(x_train, y_train, x_test, y_test):
    print((x_train.shape[0],))
    model= Sequential() 
    model.add(Dense({{choice([50,200,500])}}, input_shape=(x_train.shape[0],), activation= 'relu'))
#     model.add(Activation('relu'))
    model.add(Dropout({{uniform(0,1)}}))
    model.add(Dense({{choice([50,200,500])}},activation= 'relu'))
    #model.add(Activation('relu'))
    model.add(Dropout({{uniform(0,1)}}))
    model.add(Dense(1, activation= 'linear'))
   # model.add(Activation('linear'))
    
################################################
    def rmse(y_true, y_pred):
        from keras import backend
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
# mean squared error (mse) for regression  (only for Keras tensors)
    def mse(y_true, y_pred):
        from keras import backend
        return backend.mean(backend.square(y_pred - y_true), axis=-1)
    def r_square(y_true, y_pred):
        SS_res =  K.sum(K.square(y_true - y_pred)) 
        SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
        return ( 1 - SS_res/(SS_tot + K.epsilon()) )
#############################################

    model.compile(loss='mean_absolute_error', optimizer= 'adam', metrics=[r_square, rmse])
    from keras.utils import print_summary
    print_summary(model, line_length=None, positions=None, print_fn=None)
    result= model.fit(x_train, y_train,
                      batch_size={{choice([64,128])}},
                      epochs={{choice([50,100,150])}},
                      verbose=2,
                      validation_split =0.15)
    validation_acc= np.min(result.history['val_loss'])
    print('Lowest Validation Loss:', validation_acc)
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}   

best_run, best_model= optim.minimize(model=create_model,
                                     data=data,
                                     algo=tpe.suggest,
                                     max_evals=5,
                                     trials=Trials(),
                                     notebook_name='NeuralAnalysis')



>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:
    from keras import backend
except:
    pass

try:
    from keras import backend
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras import losses
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:


job exception: Error when checking input: expected dense_109_input to have shape (4457,) but got array with shape (7,)



  0%|                                                                            | 0/5 [00:00<?, ?trial/s, best loss=?]


ValueError: Error when checking input: expected dense_109_input to have shape (4457,) but got array with shape (7,)

In [ ]:

# if __name__=='__main__':
#     best_run, best_model= optim.minimize(model=create_model,
#                                         data=data,
#                                         algo=tpe.suggest,
#                                         max_evals=5,
#                                         trials=Trials())
#     x_train, y_train, x_test, y_test= data()
#     print('Evaluation of best performing model:')
#     print(best_model.evaluate(x_test,y_test))
#     print('Best performing model chosen hyper-parameters')
#     print(best_run)
    

#     score= model.evaluate(x_test,y_test, verbose=0)
#     accuracy= score[1]
#     return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}